# STAC, Geopandas, Xarray, Dask, Holoviz

This notebook will showcase foundational open-source Python libraries in the Pangeo stack of tools, working up from small data to datasets that excede local memory. We'll be going over this notebook at [FOSS4G 2021 Buenos Aires Workshop](https://callforpapers.2021.foss4g.org/foss4g-2021-workshop/talk/NMFGKD/). Plan to spend at least 30 minutes to 1 hour going over this notebook!


## Learning objectives:

- discover data with [STAC](https://stacspec.org) APIs
- perform basic geospatial vector operations with [Geopandas](https://geopandas.org)
- perform basic geospatial raster operations with [Xarray](http://xarray.pydata.org/en/stable/)/[Rioxarray](https://corteva.github.io/rioxarray/stable/)
- single-machine scaling with [Dask](https://dask.org)
- interactive browser visualizations with [Holoviz](https://holoviz.org)

In [ ]:
# STAC API search
import pystac_client

# Vector utilities
import geopandas as gpd

# Raster utilities
import xarray as xr
import rioxarray
import stackstac

# Visualization
import hvplot.xarray
import hvplot.pandas

# Other tools
import json
import pyproj
import planetary_computer

In [ ]:
# It's good practice to keep track of library versions
print(f'pystac_client={pystac_client.__version__}')
print(f'geopandas={gpd.__version__}')
print(f'xarray={xr.__version__}')
print(f'rioxarray={rioxarray.__version__}')
print(f'hvplot={hvplot.__version__}')

## Vector data 

Geospatial vector data consists basic geometries (Points, Lines, Polygons) with coordinate reference system information (CRS). If you're new to vector data, check out this Software Carpentry [lesson](https://carpentries-incubator.github.io/geospatial-python/).

[Pandas](https://pandas.pydata.org) is a core scientific Python library to work with "Panel Data" (PanDas). Basically if you have a spreadsheet or database you should be using Pandas! Pandas has many input/output (I/O) functions, and two core data structures - the "Series" and "DataFrame". 

[Geopandas](http://geopandas.org) extends Pandas to work efficently with collections of geographic Vector data - geometric shapes that are georeferenced to a position on Earth's surface. Geopandas data objects are, you might have guessed, called "GeoSeries" and "GeoDataFrame".

There are *many* vector formats for geospatial data. A very common one is [GeoJSON](https://gdal.org/drivers/vector/geojson.html), which can be easily represented as a Python dictionary:

In [ ]:
# Barreal, Argentina location in GeoJSON
# from https://geojson.io

area_of_interest = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          -69.466552734375,
          -31.62532121329918
          
        ]
      }
    }
  ]
}

with open('point.geojson', 'w') as f:
   json.dump(area_of_interest, f)

In [ ]:
gf_aoi = gpd.read_file('point.geojson')
gf_aoi['id'] = 'barreal'
gf_aoi

In [ ]:
# Geopandas facilitates geospatial operations such as reprojection
# https://epsg.io/32719
gf_aoi_utm = gf_aoi.to_crs('EPSG:32719')
gf_aoi_utm.buffer(100) 
# Why reproject? What are the units here?

**Summary:**

- We created a simple GeoPandas Dataframe for a POINT geometry
- Geopandas was used to reproject coordinate reference system and add a buffer to form a POLYGON

## Search for data

[SpatioTemporal Asset Catalogs (STAC)](https://stacspec.org) are a standard among imagery providers to simplify and unify search capabilities. Metadata is in JSON format and definited by a community-built standard [core specification](https://github.com/radiantearth/stac-spec) with optional [extensions](https://stac-extensions.github.io).

[pystac_client](https://github.com/stac-utils/pystac-client) is a Python client for working with STAC Catalogs and APIs. It uses [PySTAC](https://pystac.readthedocs.io) behind the scenes to navigate STAC metadata.

There are several public STAC API endpoints, which you can find on the [STAC Index Website](https://stacindex.org/catalogs?access=public&type=api), a few are listed below:

| provider | endpoint | datacenters |
| - | - | - | 
| [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) | https://planetarycomputer.microsoft.com/api/stac/v1 | Azure West Europe |
| [Element84 Earthsearch](https://www.element84.com/earth-search/) | https://earth-search.aws.element84.com/v0 | AWS multiple regions | 
| [NASA CMR STAC Cloud Proxy](https://github.com/nasa/cmr-stac) | https://cmr.earthdata.nasa.gov/cloudstac | AWS us-west-2 | 

For high-performance and cost-effective analysis, always keep in mind where data is located! For this workshop we are running on servers in Microsoft Azure’s `West Europe` region, so we'll use mostly datasets hosted in that region by the [Planetary Computer Initiative](https://planetarycomputer.microsoft.com). We run computations where the data is stored, and bring small subsets or visualizations back for download.

In [ ]:
# See documentation of all datasets at https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/
# See also data catalog website: https://planetarycomputer.microsoft.com/catalog
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

collections = catalog.get_children()
for collection in collections:
    print(f"{collection.id} - {collection.title}") 

In [ ]:
# Use the 'geometry' information from our GeoJSON
search = catalog.search(collections=["nasadem"], 
                        intersects=area_of_interest['features'][0]['geometry'], 
                       )

STAC ItemCollections can be represented as *vector* data in GeoJSON format !

In [ ]:
# A convenient way to display results is as a Geopandas GeoDataFrame
gf = gpd.GeoDataFrame.from_features(search.get_all_items_as_dict())

# BUG: 'ids' droppped https://github.com/geopandas/geopandas/pull/2003
gf['id'] = [item.id for item in search.get_all_items()]
gf

**Aside:** Above we have a simple work-around for a geopandas bug. If open-source libraries are missing some functionality you can help! In fact, the success of open source software relies on community contributions and volunteer efforts. Remember, you are not just a *user* of these tools, but a *supporter* of these tools. Check out the excellent Xarray contributing guide for ideas of how to get started contributing http://xarray.pydata.org/en/stable/contributing.html.

### Visualization of geospatial data

In this notebook we're going to illustrate the use of [hvplot](https://hvplot.holoviz.org/user_guide/Geographic_Data.html), which works very well to visualize both vector and raster data in a jupyter notebook. `hvplot` is used as an "accessor" on pandas and xarray objects.

In [ ]:
# As above we can use holoviews to plot this
dem_footprint = gf.hvplot.polygons(geo=True, alpha=0.2)
dem_footprint

In [ ]:
# Use PySTAC to iterate through STAC Items
for item in search.get_all_items():
    print(item.id)

In [ ]:
# before doing any analysis with new datasets, take a step back and acquaint yourself with the details!
collection = item.get_collection()
collection.description

In [ ]:
[p.to_dict() for p in collection.providers]

STAC metadata keeps track of data provenance, so you should always be able to get back to the original data producer or 'archive of record'. In the case of NASADEM it's NASA's LPDAAC data center https://lpdaac.usgs.gov/products/nasadem_hgtv001/. On that page you'll find important information that might not be present in STAC metadata, such as the vertical reference or 'datum' of elevation values, which is "EGM96". 

In [ ]:
# We have a single STAC Item, that can contain multiple STAC Assets:
for asset_key, asset in item.assets.items():
    print(f"{asset_key:<10} - {asset.href}")

In [ ]:
# For single Items it's sometimes easier to convert STAC JSON to a python dictionary
item.to_dict()

**Summary:**

- We found elevation data covering our point via `pystac_client` microsoft planetary computer's STAC API
- We converted the STAC search Item footprint into a Geopandas dataframe and vizualized it with hvplot.
- We navigated the STAC metadata with PySTAC and as a python dictionary

## Raster data

We've seen that [Pandas](https://pandas.pydata.org/pandas-docs/stable/) and [Geopandas](http://geopandas.org) are excellent libraries for analyzing tabular "labeled data". [Xarray](http://xarray.pydata.org/en/stable/) is designed to make it easier to work with with _labeled multidimensional data_. By _multidimensional data_ (also often called _N-dimensional_), we mean data with many independent dimensions or axes. For example, we might represent Earth's surface temperature $T$ as a three dimensional variable

$$ T(x, y, t) $$

where $x$ and $y$ are spatial dimensions and and $t$ is time. By _labeled_, we mean data that has metadata associated with it describing the names and relationships between the variables. The cartoon below shows a "data cube" schematic dataset with temperature and preciptation sharing the same three dimensions, plus longitude and latitude as auxilliary coordinates.

![xarray data model](https://github.com/pydata/xarray/raw/main/doc/_static/dataset-diagram.png)

### Xarray data structures

Like Pandas, xarray has two fundamental data structures:
* a `DataArray`, which holds a single multi-dimensional variable and its coordinates
* a `Dataset`, which holds multiple variables that potentially share the same coordinates

#### DataArray

A `DataArray` has four essential attributes:
* `values`: a `numpy.ndarray` holding the array’s values
* `dims`: dimension names for each axis (e.g., `('x', 'y', 'z')`)
* `coords`: a dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings)
* `attrs`: an `OrderedDict` to hold arbitrary metadata (attributes)

#### DataSet

A dataset is simply an object containing multiple DataArrays indexed by variable name


### RioXarray Extension

Note that Xarray is a generic nD array library and therefore does not handle geospatial-specific functionality on its own (like Coordinate Reference System (CRS) managment, reprojection, etc.). The [RioXarray extension](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html) adds that functionality! RioXarray build on top of [rasterio](https://rasterio.readthedocs.io/en/latest/), which is built on top of [GDAL](https://gdal.org), so you can see it's FOSS4G all the way down! 

In [ ]:
# IMPORTANT: we are passing a URL here rather than a local file path
da = rioxarray.open_rasterio(asset.href, masked=True)
da.name = item.id
da

In [ ]:
# When using rioxarray, we have regular xarray objects that are enhanced with a '.rio' accessor that adds functionality
da.rio.crs

In [ ]:
# Reprojection syntax is similar to geopandas
da_utm = da.rio.reproject(da.rio.estimate_utm_crs())
print(da_utm.rio.crs)
da_utm

In [ ]:
# Xarray has very convenient label-based indexing so you don't have to remember integer axis ordering

# Here we get an East-West Profile through a particular latitude
da.sel(band=1, y=-31.5, method='nearest')

In [ ]:
# We can use hvplot on xarray objects for interactive plots in the browser
# Using bokeh toolbars, resolution is updated as you zoom in!

da.hvplot.image(x='x',y='y',
                geo=True, # convey that 'x' and 'y' correspond to 'longitude, latitude'
                rasterize=True, # don't send entire array to browser, just rendered image
                cmap='viridis')

In [ ]:
# We can also plot the profile line we extracted
da.sel(band=1, y=-31.5, method='nearest').hvplot.scatter(xlabel='longitude')

In [ ]:
# Same profile line in UTM (both axes in meters)
transform = pyproj.Proj(da_utm.rio.crs)
easting, northing = transform(-70, -31.5)

In [ ]:
da_utm.sel(band=1, y=northing, method='nearest').hvplot.scatter(xlabel='easting (m)', data_aspect=1, frame_width=1000)

In [ ]:
# NOTE that if we run calculations on the entire DataArray, all data is read into local RAM and the result is presented to us:
da.mean()

In [ ]:
# Your turn! 

# - Pick a different point in Argentina and make some plots of elevation
# - Try some different plots 

## Dask


[Dask](https://docs.dask.org/en/latest/) is a flexible parallel computing library for analytic computing. Dask provides dynamic parallel task scheduling and high-level big-data collections like `dask.array` and `dask.dataframe`. 

Dask is tightly integrated with Xarray, such that parallel computations can happen with very little change to your code. Often, you only need to specify dask "chunks", which describe how an array is split up over many sub-arrays.

![Dask Arrays](http://dask.pydata.org/en/latest/_images/dask-array-black-text.svg)
_source: [Dask Array Documentation](http://dask.pydata.org/en/latest/array-overview.html)_

In [ ]:
# Although the array size is small, we can use our elevation example to understand how dask integrates with xarray
print(type(da.data))
da = da.chunk(dict(x=2048,y=2048))
print(type(da.data))
da

Note that xarray now provides us with a nice HTML representation of the underlying dask array and how it is divided into 4 chunks. Often the best performance is obtained when chunks match data storage. NASADEM is stored as a Cloud Optimized Geotiff or ['COG'](https://www.cogeo.org), which typically are stored as 512x512 tiles. If you really want to dig into performance, have a look through [these notebooks](https://github.com/pangeo-data/cog-best-practices), but it's usually okay to pick chunk sizes that are a multiple of the data tile size (512*n)

In [ ]:
# Let's again just look at the mean of all the pixels
da.mean()

**What happened?**

now that our DataArray has dask arrays, only the 'task graph' of computations is created, and those computations are not triggered until explicitly asked for or visualized. This is known as 'lazy computation'. You can trigger the computation with a `.compute()` call

In [ ]:
da.mean().compute()

**Summary**

- We converted our array into a dask-array and let dask manage a parallel computation for us on our raster divided into 4 chunks
- By default, dask uses its [single-machine multi-threaded scheduler](https://docs.dask.org/en/latest/setup/single-machine.html), which distributes work across multiple local CPU threads.

### Dask clusters

Dask needs a collection of computing resources in order to perform parallel computations. Dask Clusters have different names corresponding to different computing environments (for example, [LocalCluster](https://distributed.dask.org/en/latest/local-cluster.html) for a single machine, [PBSCluster](http://jobqueue.dask.org/) for your HPC, or [Kubernetes Cluster](http://kubernetes.dask.org/) for machines distributed on the Cloud). Each cluster has a certain number of computing resources called 'Workers', that each get allocated CPU and RAM. The dask scheduling system maps jobs to each worker on a cluster for you, so the syntax is mostly the same once you initialize a cluster!

The key advantage of explicitly creating a cluster is that it gives you visual dashboard [diagnostics](https://docs.dask.org/en/latest/diagnostics-distributed.html) (either on a standalone 'Dashboard' webpage or visualized in JupyterLab with the [Dask Labextension](https://github.com/dask/dask-labextension) ).

In [ ]:
#Let's start simple with a LocalCluster that makes use of all the cores and RAM we have on a single machine
# NOTE: by default this cluster will use multiple 'processes' instead of threads by default
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()

# explicitly connect to the cluster we just created
client = Client(cluster)
client

In [ ]:
# Now calculate the standard deviation and watch the diagnotic dashboard 'task stream' 
da.std().compute()

## Scaling on a single machine

Let's use our LocalCluster to do some analysis on a dataset that would otherwise excede our 16GB of memory. Dask will ensure that we don't hit out of memory errors as it crunches through our computation on multiple threads. For illustration we'll use a STAC of Landsat8 scenes... p.s. Landsat9 Successfully [launched yesterday 9/27/2021](https://www.nasa.gov/press-release/nasa-launches-new-mission-to-monitor-earth-s-landscapes)!! 

There are a lot of Landsat archives out there with subtle differences, note this dataset is coming from the US Geological Survey and the original archive is hosted on Amazon Web Services in a requester-pays bucket
https://www.usgs.gov/core-science-systems/nli/landsat/landsat-commercial-cloud-data-access?qt-science_support_page_related_con=1#qt-science_support_page_related_con 

In [ ]:
# Check out the full documentation at https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2

search = catalog.search(collections=["landsat-8-c2-l2"], 
                        intersects=area_of_interest['features'][0]['geometry'], 
                        datetime=['2015-01-01T00:00:00Z', None],
                        # Only keep 'Tier1' data 'suitable for timeseries analysis'
                        query=['landsat:collection_category=T1', 'landsat:wrs_path=232'], 
                       )

In [ ]:
# A convenient way to display results is as a Geopandas GeoDataFrame
gf = gpd.GeoDataFrame.from_features(search.get_all_items_as_dict())
print(len(gf))
# BUG: 'ids' droppped https://github.com/geopandas/geopandas/pull/2003
gf['id'] = [item.id for item in search.get_all_items()]
gf.head()

In [ ]:
# Composing holoviz elements in single visualization (basemap tiles, polygons and point)
polygons = gf.hvplot.polygons(geo=True, 
                   tiles='EsriTerrain', 
                   hover_cols=['landsat:wrs_path','landsat:wrs_row'],
                   alpha=0.2)

polygons * dem_footprint * gf_aoi.hvplot.points(geo=True)

In [ ]:
# To read this data, we need to sign urls as described here 
# https://planetarycomputer.microsoft.com/docs/concepts/sas/
items = search.get_all_items()

signed_items = []
for item in items:
    item = item.clone()
    # Fix metadata issue https://github.com/stactools-packages/landsat/issues/12
    item.properties['proj:epsg']=32619
    item.clear_links()
    signed_items.append(planetary_computer.sign(item))

In [ ]:
# We can open a single asset as before
item = signed_items[0]
da = rioxarray.open_rasterio(item.assets['SR_B4'].href, masked=True)
da.sel(band=1).hvplot(x='x',y='y',rasterize=True)
da.rio.crs

In [ ]:
da.hvplot.image(x='x', y='y', rasterize=True,
                data_aspect=1,
                cmap='reds')

In [ ]:
# Hmmm... why are the y-axis units negative? 
# https://www.usgs.gov/faqs/why-do-landsat-scenes-southern-hemisphere-display-negative-utm-values?qt-news_science_products=0#qt-news_science_products

In [ ]:
# Stackstac is a powerful library for converting many STAC Items to an xarray dataarray

daL8 = stackstac.stack(
    [x.to_dict() for x in signed_items], assets=["SR_B4", "SR_B5"],
)
daL8.name = 'landsat-8-c2-l2'
daL8

Don't be scared by the huge *total* size of this dataset! That represents the uncompressed size of *all* 2D rasters, but often we just want a subset, and we can use Xarray's API + dask to retrieve a subset efficiently:

In [ ]:
# For example, let's use a small range of latitude and longitude
transform = pyproj.Proj(da.rio.crs)
left, top = transform(-69.8, -31.5)
right, bottom = transform(-69.3, -32)
print(left,right,bottom,top)

In [ ]:
subset = daL8.sel(x=slice(left,right), y=slice(top,bottom)) # note top,bottom b/c y coords stored in *decreasing* order
DS = subset.to_dataset(dim='band')
DS

In [ ]:
# Compute NDVI 
NDVI = (DS.SR_B5 - DS.SR_B4) / (DS.SR_B5 + DS.SR_B4)

In [ ]:
# Plot interactive visualization
result = NDVI.persist() # pull into cluster memory, watch the task graph to see work happening
result.hvplot.image(x='x',y='y',
                    rasterize=True, 
                    data_aspect=1, frame_width=400,
                    cmap='BrBG', clim=(-1,1), 
                    widget_location='bottom')

In [ ]:
time_series = result.sel(x=slice(4.52e5,4.56e5), y=slice(-3.500e6, -3.501e6)).mean(dim=['x','y']).compute()
time_series.name = 'NDVI'
time_series.hvplot.scatter(x='time')

**Keep going!**
- examine some quality bands and use them to mask your results
- look at different indices such as EVI https://www.usgs.gov/core-science-systems/nli/landsat/landsat-enhanced-vegetation-index?qt-science_support_page_related_con=0#qt-science_support_page_related_con or NDSI
- mask or look a NDVI statistics versus elevation with NASADEM
- use xarray advanced interpolation to extract values at many points
- compute mean NDVI and save resulting 2D geotiff
- bring in an open dataset in a different cloud data center like Copernicus DEM https://registry.opendata.aws/copernicus-dem/ 
- dig into the dask diagnotics and experiment with chunk settings for your data and workflow 

## Learn More

There are lots of great tutorials out there to continue to explore.

- The Official Documentation for each Library
- [Pangeo Gallery Tutorial](https://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/)
- [Earthlab Data Science Course](https://www.earthdatascience.org)